# DataFrames.jl @ 0.22.2

最近Juliaでデータフレームを操作するパッケージ[DataFrames.jl](https://github.com/JuliaData/DataFrames.jl/releases/tag/v0.22.0)のバージョンが少しbump upしました。ただし，この執筆時点での最新バージョンは0.22.2です。

そもそもデータフレームとは，基本的に二次元配列の構造で，行列と同様に行と列を持ちます。

多言語だとRのdplyrやPythonのpandasなどがありますが，JuliaではDataFrames.jlがデータフレームの操作を実行するためのパッケージとなります。連携するパッケージとしては`DataFramesMeta.jl`や`Query`などがありますが，それらを使わなくても，たいていのことはできる印象です。

Juliaの公式パッケージはGutHub上で管理されているので，GutHubで変更履歴や，おもな変更点を確認することができます。

どうやら今回のおもな変更点はデータフレームの列操作に関する関数`select`/`transform`の改善や，データの持ち方を縦から横に変更する関数`unstack`に関する挙動のようです。

これに加えて，いくつかの補助関数(`isapprox`, `empty`)が新たに追加されました。

In [1]:
using Pkg; Pkg.add(name = "DataFrames", version = "0.22.0");

   Updating registry at `~/.julia/registries/General`
       Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [2]:
using DataFrames;

---

## 1. 基本操作

### 1.1 データフレームをつくる

`DataFrame(列名 = 変数)`という作り方は確認済みなので，行列からデータフレームを作る方法をチェックしておきます。

1. `DataFrame(行列)`
2. `DataFrame(行列, [シンボルor文字列のベクトル])`
3. `DataFrame(辞書)`
4. `DataFrame(列名 => 要素)`

４つ目の書き方はペア`Pair`を利用した書き方です。気をつけなければならないのは，辞書やペアを利用する場合はすべての要素の大きさが同じである必要があります。

In [7]:
mat = randn(3, 3);
A = DataFrame(mat, [:a, :b, :c])

,a,b,c
,Float64,Float64,Float64
1,0.184904,-0.286792,0.0563817
2,-0.276054,-1.15357,0.0512511
3,0.306679,1.14489,-0.656571


In [9]:
DataFrame(:a => 1:2, :b => 2:4) # 長さが違うので，エラーになる。

LoadError: DimensionMismatch("column :a has length 2 and column :b has length 3")

---

### 1.2 データフレームの要素を取り出す

#### 取り出す

作ったデータフレームの列要素のアクセスは，データフレームを`A`，取り出したい列名を`a`とすると，

- `A.a`
- `A."a"`
- `A[!, :A]`
- `A[!, "A"]`
- `getproperty(A, :a)`
- `getproperty(A, "a")`

で取り出すことができます。`[]`や`getproperty`を使用する場合，シンボルでも文字列でもどちらでもアクセス可能です（シンボルのほうがわずかに効率がいいらしい）。

これらの方法で取り出された配列はコピーされません。コピーされないということは取り出した要素を変化させると，もとのデータフレームも変化するということです。コピーを取り出したい場合は`A[:, :a]`のように，`!`の代わりに`:`を使用します。

複数の列にアクセスすることもできます。

In [10]:
A[:, [:a, :b]]

,a,b
,Float64,Float64
1,0.0951098,2.2853
2,0.902059,-0.80384
3,-1.21661,-1.04741


### 1.3 正規表現を使って要素を指定する

正規表現を使ってマッチする列を取り出すこともできます。正規表現を使う場合には`r""`をつかって`Regex`を作り出します。

正規表現による強力なマッチング機能が利用できるのは，DataFrames.jlの強みだと思います。

In [11]:
A[:, r"[ab]"]

,a,b
,Float64,Float64
1,0.0951098,2.2853
2,0.902059,-0.80384
3,-1.21661,-1.04741




サポート関数を使っても，列方向に対する操作ができます。利用できるサポート関数は

- `Not()`   選択したシンボルか`Regex`にマッチした列**以外**を取り出す。
- `Cols()`  選択したシンボルか`Regex`にマッチした列**だけ**を取り出す。
- `All()`   選んでくる条件は`Cols()`と同じ。
- `Between(first, last)`    `first`~`last`間の列をすべて取り出す。

です。

`Cols`と`All()`は機能的には同一ですが，`Cols()`は何も列を選択しないことができるので，こちらのほうができることがひとつ多い印象。

**並び替える**

これらのサポート関数は，列方向に関する並び替えを行いたいときに役に立ちます。並び替えには`Regex`とシンボル，文字列が使えます。つまり，列選択のときと全く同じです。

In [12]:
A[!, Cols(r"a|c", "b")]

,a,c,b
,Float64,Float64,Float64
1,0.0951098,-1.12006,2.2853
2,0.902059,-0.376344,-0.80384
3,-1.21661,-0.461952,-1.04741


### 1.4 列を指定して，変更する

列の要素を指定する方法は他にもあります。それが`select()`と`transform()`です。しかし，この２種類の関数は，ただ取り出すだけでなく，指定した要素に対して関数を作用させるための関数です。

- `select()`, `select!()`   選択した列**だけ**を取り出す
- `transform()`, `transform!()` 選択した列**以外も**取り出す。

`!`がついている方はin-place関数であり，作用させた変数の内容そのものを破壊的に変更します。

`transform`は選択列以外も取り出すので「なんのこっちゃ」となるかもしれませんが，この２つの関数は，ただ列を取り出すだけでなく，取り出した列に対して関数を作用させるための関数であるため，`transform`が重要になります。

これらは`Regex`やシンボル，文字列と組み合わせて使うことができるだけでなく，先述したサポート関数と一緒に使うこともできます。

**`select()`**

マッチした列だけを取り出して，新しいデータフレームを作成します。

さらに，単純に取り出すだけでなく，列名を変更したり，取り出した列に関数を作用させて新しい列にすることもできます。`select`はデフォルトでは自動的にコピーを作成する仕様になっていますが，`..., copycols = false`でコピーの作成をしないようにすることもできます。

In [13]:
B = select(A, :a, copycols = true)
C = select(A, :a, copycols = false)

,a
,Float64
1,0.0951098
2,0.902059
3,-1.21661


In [14]:
B.a === A.a

false

In [20]:
C.a === A.a

true

`select()`は単に列を取り出すだけでなく，選択した列に対して関数を適用し，それを新たな列として返すという操作もできます。

新しい列を作るための書き方は，

`select(A, :列名 => 関数 => :新しい列名)`

です。関数には`()`でくくることで無名関数も使うことができます。

In [12]:
select(A, :a => (x -> 2x) => :d)

,d
,Float64
1,0.369807
2,-0.552108
3,0.613357


関数は列の要素に対してまとめててきようされます。そのため行の各要素に関数を作用させたいときはbroadcastを利用します.

例えば各要素を`round`したいときは，

```
select(A, :a => round => :d)
```

ではなく

In [14]:
select(A, :a => (x -> round.(x)) => :d)

,d
,Float64
1,0.0
2,-0.0
3,0.0


と実行する必要があります。

**`transform()`**

`select()`と同じ記述方法で列を指定しますが，`select()`とは異なり，選択していない列も残して，値を返します。

In [8]:
transform(A, :a => (x -> 2x) => :d)

,a,b,c,d
,Float64,Float64,Float64,Float64
1,0.184904,-0.286792,0.0563817,0.369807
2,-0.276054,-1.15357,0.0512511,-0.552108
3,0.306679,1.14489,-0.656571,0.613357


### 1.6 行方向の操作を実行する。

`select`や`transform`では基本的に列方向に対して関数を作用させました。しかしデータフレームには行ごとに意味をもたせたデータを入力したいこともあります（たとえばあるクラスのテスト得点の表とか）。

そんな操作をするためには`ByRow`を使うと良いです。

In [19]:
B = DataFrame("名前" => ["Kato", "Noguchi", "Yamada"], "国語" => [20, 31, 55], "数学" => [100, 23, 78], "英語" => [10, 30, 89])

,名前,国語,数学,英語
,String,Int64,Int64,Int64
1,Kato,20,100,10
2,Noguchi,31,23,30
3,Yamada,55,78,89


In [24]:
transform(B, Not(:名前) => ByRow(＋) => :合計)

,名前,国語,数学,英語,合計
,String,Int64,Int64,Int64,Int64
1,Kato,20,100,10,130
2,Noguchi,31,23,30,84
3,Yamada,55,78,89,222


**行方向の取り出し**

列方向のデータフレーム操作をみたところで，単純な取り出しの例についても確認しておきましょう。

列方向だけでなく，行方向に対しても一部の要素を取り出すことができます。`[,]`のカンマの左側に，ベクトルなどで位置を指定してやることで，取り出せます。

In [48]:
A[1:2, :]

,a,b
,Float64,Float64
1,0.0951098,2.2853
2,0.902059,-0.80384


特定の条件にマッチする行だけを取り出したい場合には，２通りの書き方ができます。

- `A[条件, :]`
- `filter(:ID -> 関数, A)`

`filter()`は結構便利な関数です。次のように派生した書き方ができます。

In [50]:
filter(:a => i -> i < 0, A)

,a,b,c
,Float64,Float64,Float64
1,-1.21661,-1.04741,-0.461952


### 1.7 `DataFramesRow`には要注意

しかし，１行だけ取り出す場合，`DataFrame`から`DataFramesRow`と呼ばれる型に変化します。`DataFrame`の型を維持したままで１行だけ取り出す場合は，`A[[1], :a]`のように，ベクトルで行の要素を指定すればよいです。

`DataFramesRow`に対してはbroadcastをすることができません。

In [63]:
A[1, [:a, :b]] # DataFramesRow

,a,b
,String,Float64
1,1,1.0


In [65]:
A[[1], [:a, :b]] # DataFrame

,a,b
,String,Float64
1,1,1.0


In [69]:
string.(A[1, [:a, :b]])

LoadError: ArgumentError: broadcasting over `DataFrameRow`s is reserved

このように`DataFramesRow`に対してはbroadcast演算をすることはできません。

---

##　2. 結合する

データフレーム操作で頻繁に行われるのが，データフレームの結合です。単純に行や列の大きさが同じものをくっつけるだけでなく，特定列の要素をキーにマッチさせたりすることができます。

こうした操作は`**join(A, B)`系の関数で実行できます。

### 2.1 `**join()`の基本的な書き方

基本的な使用方法は

```
**join(A, B, on = :列名)
```

です。AとBで列名が違っている場合，

```
**join(A, B, on = :Aの列名 => :Bの列名)
```

というように，`Pair`を使用します。複数のキーがある場合はシンボルやペアをベクトルとして渡してあげます。

```
**join(A, B, on = [:列名1, :Aの列名２ => :Bの列名２])
```

### 2.2 joinの種類

- `leftjoin`, `rightjoin` 左右どちらかのデータフレーム(A)を基準としてマッチさせる。基準のデータフレームAにないBの行は無視され，Bの列のうちAに含まれていないものは欠測`missing`として追加される。
- `innerjoin` 両方のデータフレームの要素を行に関してすべて残す形でマッチさせる。片方のデータフレームにしかない行も完全に保存される。
- `outerjoin` 両方のデータフレームの要素のうち，キーがマッチした行だけ残し，それ以外は無視する。この形では，マッチによる欠測が生じない。
- `semijoin` マッチした行だけを保存するが，さらに列に関しては基準となるAの列だけを保存し，Bに関する要素は完全に無視される。
- `antijoin` マッチした行**以外**を保存するが，列に関しては基準となるAの列だけが保存される。`semijoin`の逆バージョン。
- `crossjoin` 与えたデータフレームらの直積を返す。マッチと言うよりも，全パタンの網羅するための関数？

実際の実行例は[パッケージドキュメント](https://dataframes.juliadata.org/stable/man/joins/)に詳しいです。

In [25]:
people = DataFrame(ID = [20, 40], Name = ["John Doe", "Jane Doe"]);
jobs = DataFrame(ID = [20, 40], Job = ["Lawyer", "Doctor"]);

,ID,Job
,Int64,String
1,20,Lawyer
2,40,Doctor


In [29]:
crossjoin(people, jobs; makeunique = true)

,ID,Name,ID_1,Job
,Int64,String,Int64,String
1,20,John Doe,20,Lawyer
2,20,John Doe,40,Doctor
3,40,Jane Doe,20,Lawyer
4,40,Jane Doe,40,Doctor


## 3 データフレームを変形させる

データフレームをピボットしたり，変形したりする操作もDataFrames.jlの標準的な関数として提供されています。それが`stack`と`unstack`です。
データフレームを縦方向に伸ばす（要素を縦に積み上げる）のが`stack`であり，逆に横方向に伸ばす（積み上がっている要素を横に展開する）のが`unstack`です。




In [30]:
?unstack

search: unstack RoundNearestTiesAway



```
unstack(df::AbstractDataFrame, rowkeys, colkey, value; renamecols::Function=identity,
        allowmissing::Bool=false, allowduplicates::Bool=false)
unstack(df::AbstractDataFrame, colkey, value; renamecols::Function=identity,
        allowmissing::Bool=false, allowduplicates::Bool=false)
unstack(df::AbstractDataFrame; renamecols::Function=identity,
        allowmissing::Bool=false, allowduplicates::Bool=false)
```

Unstack data frame `df`, i.e. convert it from long to wide format.

Row and column keys will be ordered in the order of their first appearance.

# Positional arguments

  * `df` : the AbstractDataFrame to be unstacked
  * `rowkeys` : the columns with a unique key for each row, if not given, find a key by grouping on anything not a `colkey` or `value`. Can be any column selector (`Symbol`, string or integer; `:`, `Cols`, `All`, `Between`, `Not`, a regular expression, or a vector of `Symbol`s, strings or integers).
  * `colkey` : the column (`Symbol`, string or integer) holding the column names in wide format, defaults to `:variable`
  * `value` : the value column (`Symbol`, string or integer), defaults to `:value`

# Keyword arguments

  * `renamecols`: a function called on each unique value in `colkey`; it must return the name of the column to be created (typically as a string or a `Symbol`). Duplicates in resulting names when converted to `Symbol` are not allowed. By default no transformation is performed.
  * `allowmissing`: if `false` (the default) then an error will be thrown if `colkey` contains `missing` values; if `true` then a column referring to `missing` value will be created.
  * allowduplicates`: if`false`(the default) then an error an error will be thrown if combination of`rowkeys`and`colkey`contains duplicate entries; if`true`then  then the last encountered`value` will be retained.

# Examples

```julia
julia> using Random

julia> Random.seed!(1234);

julia> wide = DataFrame(id = 1:6,
                        a  = repeat([1:3;], inner = [2]),
                        b  = repeat([1:2;], inner = [3]),
                        c  = randn(6),
                        d  = randn(6))
6×5 DataFrame
 Row │ id     a      b      c          d
     │ Int64  Int64  Int64  Float64    Float64
─────┼────────────────────────────────────────────
   1 │     1      1      1   0.867347   0.532813
   2 │     2      1      1  -0.901744  -0.271735
   3 │     3      2      1  -0.494479   0.502334
   4 │     4      2      2  -0.902914  -0.516984
   5 │     5      3      2   0.864401  -0.560501
   6 │     6      3      2   2.21188   -0.0192918

julia> long = stack(wide)
12×5 DataFrame
 Row │ id     a      b      variable  value
     │ Int64  Int64  Int64  String    Float64
─────┼───────────────────────────────────────────
   1 │     1      1      1  c          0.867347
   2 │     2      1      1  c         -0.901744
   3 │     3      2      1  c         -0.494479
   4 │     4      2      2  c         -0.902914
  ⋮  │   ⋮      ⋮      ⋮       ⋮          ⋮
  10 │     4      2      2  d         -0.516984
  11 │     5      3      2  d         -0.560501
  12 │     6      3      2  d         -0.0192918
                                   5 rows omitted

julia> unstack(long)
6×5 DataFrame
 Row │ id     a      b      c          d
     │ Int64  Int64  Int64  Float64?   Float64?
─────┼────────────────────────────────────────────
   1 │     1      1      1   0.867347   0.532813
   2 │     2      1      1  -0.901744  -0.271735
   3 │     3      2      1  -0.494479   0.502334
   4 │     4      2      2  -0.902914  -0.516984
   5 │     5      3      2   0.864401  -0.560501
   6 │     6      3      2   2.21188   -0.0192918

julia> unstack(long, :variable, :value)
6×5 DataFrame
 Row │ id     a      b      c          d
     │ Int64  Int64  Int64  Float64?   Float64?
─────┼────────────────────────────────────────────
   1 │     1      1      1   0.867347   0.532813
   2 │     2      1      1  -0.901744  -0.271735
   3 │     3      2      1  -0.494479   0.502334
   4 │     4      2      2  -0.902914  -0.516984
   5 │     5      3      2   0.864401  -0.560501
   6 │     6      3      2   2.21188   -0.0192918

julia> unstack(long, :id, :variable, :value)
6×3 DataFrame
 Row │ id     c          d
     │ Int64  Float64?   Float64?
─────┼──────────────────────────────
   1 │     1   0.867347   0.532813
   2 │     2  -0.901744  -0.271735
   3 │     3  -0.494479   0.502334
   4 │     4  -0.902914  -0.516984
   5 │     5   0.864401  -0.560501
   6 │     6   2.21188   -0.0192918

julia> unstack(long, [:id, :a], :variable, :value)
6×4 DataFrame
 Row │ id     a      c          d
     │ Int64  Int64  Float64?   Float64?
─────┼─────────────────────────────────────
   1 │     1      1   0.867347   0.532813
   2 │     2      1  -0.901744  -0.271735
   3 │     3      2  -0.494479   0.502334
   4 │     4      2  -0.902914  -0.516984
   5 │     5      3   0.864401  -0.560501
   6 │     6      3   2.21188   -0.0192918

julia> unstack(long, :id, :variable, :value, renamecols=x->Symbol(:_, x))
6×3 DataFrame
 Row │ id     _c         _d
     │ Int64  Float64?   Float64?
─────┼──────────────────────────────
   1 │     1   0.867347   0.532813
   2 │     2  -0.901744  -0.271735
   3 │     3  -0.494479   0.502334
   4 │     4  -0.902914  -0.516984
   5 │     5   0.864401  -0.560501
   6 │     6   2.21188   -0.0192918
```

Note that there are some differences between the widened results above.
